# Caso Práctico - Visualización de Datos
---
Visualización de Datos del Máster Universitario en Ingeniería y Ciencia de Datos (2023/2024)

Alumno: **Javier Orive Soto**


# 1. Importación de dependencias

In [1]:
import pandas as pd
import panel as pn
pn.extension('vega')

import numpy as np
import altair as alt

import datetime as dt

# 2. Importación de datos

## 2.1 Datos de generación

### 2.1.1. Grafico de generacion por tecnología (días)

In [62]:
# Cargar el archivo Excel
#file_path = 'Datos/Economico/PrecioMedioHorarioFinal_2023_d.xls'
file_path = 'Datos/Generacion/GeneracionTotal_2023_h.csv'
#df = pd.read_excel(file_path)
df_G = pd.read_csv(file_path, delimiter=';')

# Seleccionar solo las columnas de interés
#df = df[['name', 'value', 'datetime']]

In [63]:
# Convertir la columna datetime a tipo datetime
df_G['datetime'] = pd.to_datetime(df_G['datetime'], utc = True) #, utc = True

# Convertir la columna value a tipo float
df_G['value'] = df_G['value'].astype(float)

# Convertir la columna name a tipo categoría
df_G['name'] = df_G['name'].astype('category')

# Elimino las columnas vacías
df_G = df_G.drop(['geoid', 'geoname'], axis=1)

In [ ]:
df_G = df_G.drop(df_G[df_G['id']== 10195].index)
df_G['name'] = df_G['name'].str.slice(18)
df_G['date'] = df_G['datetime'].dt.date

df_G['year'] = df_G['datetime'].dt.year
df_G['month'] = df_G['datetime'].dt.month
df_G['day'] = df_G['datetime'].dt.day
df_G['hour'] = df_G['datetime'].dt.hour

In [65]:
df_G.head()

,id,name,value,datetime,date
4321,1169,Biogás,83.937,2022-12-31 23:00:00+00:00,2022-12-31
4322,1169,Biogás,81.609,2023-01-01 00:00:00+00:00,2023-01-01
4323,1169,Biogás,80.939,2023-01-01 01:00:00+00:00,2023-01-01
4324,1169,Biogás,80.873,2023-01-01 02:00:00+00:00,2023-01-01
4325,1169,Biogás,80.765,2023-01-01 03:00:00+00:00,2023-01-01


In [66]:
df_G.info()

<class 'pandas.core.frame.DataFrame'>
Index: 155473 entries, 4321 to 164210
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype              
---  ------    --------------   -----              
 0   id        155473 non-null  int64              
 1   name      155473 non-null  object             
 2   value     155473 non-null  float64            
 3   datetime  155473 non-null  datetime64[ns, UTC]
 4   date      155473 non-null  object             
dtypes: datetime64[ns, UTC](1), float64(1), int64(1), object(2)
memory usage: 7.1+ MB


In [72]:
# https://towardsdatascience.com/how-to-build-a-time-series-dashboard-in-python-with-panel-altair-and-a-jupyter-notebook-c0ed40f02289
date_slider = pn.widgets.DateSlider(name='Date Slider', start=dt.datetime(2023, 1, 1), end=dt.datetime(2023, 12, 31), value=dt.datetime(2023, 1, 1))

date_slider

BokehModel(combine_events=True, render_bundle={'docs_json': {'b7e7f4af-a29c-489c-a3e7-9982de82a395': {'version…

In [49]:
@pn.depends(date_slider.param.value)
def get_plot(date):
     # Load and format the data
     df = df_G # define df
     ##df[‘date’] = pd.to_datetime(df[‘date’])
     # create date filter using values from the range slider
     # store the first and last date range slider value in a var
     start_date = date_slider.value
     # create filter mask for the dataframe
     mask = (df['date'] == start_date)
     df = df.loc[mask] # filter the dataframe

     selection = alt.selection_point(fields=['name'], bind='legend')

     # create the Altair chart object
     chart = alt.Chart(df).mark_area(interpolate='step').encode(
               x=alt.X('datetime:T', title='Hora del día'),  # Eje X: Hora del día
               y=alt.Y('sum(value):Q', title='Potencia eléctrica (MW)', scale=alt.Scale(domain=[0, 42000])),  # Suma de potencia
               color=alt.Color('name:N', title='Tecnología'),  # Diferenciar por tecnología
               opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
               ).properties(
               title='Generación eléctrica por tecnología',
               width=700,
               height=400
               ).add_params(selection)

     return chart

In [50]:
title = '## Desglose de generación eléctrica'
subtitle = 'Este dashboard permite al usuario seleccionar\n el día para el que se quieren visualizar los datos.\n\n Además, también se puede seleccionar la tecnología\n para resaltarla en el gráfico.'

dashboard = pn.Row(get_plot,
                   pn.Column(title, subtitle, date_slider)
)

NameError: name 'df_G' is not defined

In [70]:
dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'fdf9fb3c-b785-483c-8daa-48501130fbe3': {'version…

### 2.1.2. Grafico de media anual generacion por tecnología

In [82]:
df_aux = df_G.groupby('name')['value'].sum().sort_values(ascending=False).reset_index()

# Calcular la suma total de la columna 'value'
total_sum = df_aux['value'].sum()

# Normalizar la columna 'value' a un 100%
df_aux['value_normalized'] = (df_aux['value'] / total_sum)

In [83]:
df_aux

,name,value,value_normalized
0,Eólica terrestre,6.108076e+07,2.419517e-01
1,Nuclear,5.412415e+07,2.143953e-01
2,Ciclo combinado,3.920130e+07,1.552832e-01
3,Solar fotovoltaica,3.648300e+07,1.445156e-01
4,Hidráulica UGH,2.068712e+07,8.194533e-02
5,Gas Natural Cogeneración,1.653210e+07,6.548655e-02
6,Turbinación bombeo,5.184742e+06,2.053767e-02
7,Solar térmica,4.673006e+06,1.851060e-02
8,Hidráulica no UGH,4.547275e+06,1.801255e-02
9,Hulla antracita,2.933010e+06,1.161817e-02


In [88]:
selection = alt.selection_point(fields=['name'], bind='legend')

# Base del gráfico
base = alt.Chart(df_aux).transform_window(
    rank='rank(value)',
    sort=[alt.SortField('value', order='descending')]
).encode(
    alt.Theta("value:Q", sort=df_aux['name'].to_list()).stack(True),
    alt.Radius("value").scale(type="sqrt", zero=True),
    color=alt.Color("name:N", title='Tecnología'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).add_params(selection).properties(
    title='Generación eléctrica anual por tecnología',
    width=400,
    height=400
)

# Gráfico de arco
c1 = base.mark_arc(innerRadius=20, stroke="#fff")

# Gráfico de texto con formato
c2 = base.mark_text(radiusOffset=10).encode(
    text=alt.Text("value_normalized:Q", format=".0%")
    ).transform_filter(
    alt.datum.rank <= 6
)

# Mostrar el gráfico
c1 + c2

alt.LayerChart(...)

In [76]:
selection = alt.selection_point(fields=['name'], bind='legend')

base = alt.Chart(df_aux).encode(
    alt.Theta("value:Q", sort=df_aux['name'].to_list()).stack(True),
    alt.Radius("value").scale(type="sqrt", zero=True),
    color=alt.Color("name:N", title='Tecnología'),
    #color="name:N",
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).add_params(selection
).properties(
title='Generación eléctrica por tecnología',
width=400,
height=300)

c1 = base.mark_arc(innerRadius=20, stroke="#fff")

c2 = base.mark_text(radiusOffset=10).encode(text="value_normalized")

c1 + c2

alt.LayerChart(...)

## 2.2 Datos económicos

### 2.2.1. Desglose de precio

In [32]:
# Cargar el archivo Excel
file_path = 'Datos/Economico/PrecioMedioHorarioFinal_2023_h.xlsx'
df_E_es = pd.read_excel(file_path)

# Seleccionar solo las columnas de interés
df_E_es = df_E_es[['id', 'name', 'value', 'datetime']]

In [33]:
# Convertir la columna datetime a tipo datetime
df_E_es['datetime'] = pd.to_datetime(df_E_es['datetime'], utc = True) #, utc = True

# Convertir la columna name a tipo categoría
df_E_es['name'] = df_E_es['name'].astype('category')

In [34]:
df_E_es = df_E_es.drop(df_E_es[df_E_es['id']== 10211].index)
df_E_es['name'] = df_E_es['name'].str.slice(32).str.capitalize()
df_E_es['date'] = df_E_es['datetime'].dt.date

In [35]:
df_E_es#['name'].unique()

,id,name,value,datetime,date
8760,811,Banda secundaria,4.64,2022-12-31 23:00:00+00:00,2022-12-31
8761,811,Banda secundaria,6.25,2023-01-01 00:00:00+00:00,2023-01-01
8762,811,Banda secundaria,6.98,2023-01-01 01:00:00+00:00,2023-01-01
8763,811,Banda secundaria,5.26,2023-01-01 02:00:00+00:00,2023-01-01
8764,811,Banda secundaria,2.80,2023-01-01 03:00:00+00:00,2023-01-01
...,...,...,...,...,...
148915,1277,Servicio de interrumpibilidad,0.00,2023-12-31 18:00:00+00:00,2023-12-31
148916,1277,Servicio de interrumpibilidad,0.00,2023-12-31 19:00:00+00:00,2023-12-31
148917,1277,Servicio de interrumpibilidad,0.00,2023-12-31 20:00:00+00:00,2023-12-31
148918,1277,Servicio de interrumpibilidad,0.00,2023-12-31 21:00:00+00:00,2023-12-31


In [51]:
@pn.depends(date_slider.param.value)
def get_plot_price(date):
     # Load and format the data
     df = df_E_es # define df
     ##df[‘date’] = pd.to_datetime(df[‘date’])
     # create date filter using values from the range slider
     # store the first and last date range slider value in a var
     start_date = date_slider.value
     # create filter mask for the dataframe
     mask = (df['date'] == start_date)
     df = df.loc[mask] # filter the dataframe

     selection = alt.selection_point(fields=['name'], bind='legend')

     # create the Altair chart object
     chart = alt.Chart(df).mark_area(interpolate='step').encode(
               x=alt.X('datetime:T', title='Hora del día'),  # Eje X: Hora del día
               y=alt.Y('value:Q', title='Precio por MWh (€/MWh)', scale=alt.Scale(domain=[-10, 250])),
               color=alt.Color('name:N', title='Concepto'),  # Diferenciar por tecnología
               opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
               order=alt.Order('sum(value):Q', sort='descending')
               ).properties(
               title='Precio electricidad',
               width=700,
               height=400
               ).add_params(selection)

     return chart

In [52]:
title = '## Desglose del precio de la electricidad'
subtitle = 'Este dashboard permite al usuario seleccionar\n el día para el que se quieren visualizar los datos.\n\n Además, también se puede seleccionar la tecnología\n para resaltarla en el gráfico.'

dashboard = pn.Row(get_plot_price,
                   pn.Column(title, subtitle, date_slider))

/usuarios/jos/.conda/envs/jos_prueba/lib/python3.8/site-packages/panel/viewable.py:298: ParamFutureWarning: Parameter 'object' on <class 'panel.pane.base.PaneBase'> is being given a valid parameter reference <function get_plot_price at 0x7fcddb022160> but is implicitly allow_refs=False. In future allow_refs will be enabled by default and the reference <function get_plot_price at 0x7fcddb022160> will be resolved to its underlying value alt.Chart(...). Please explicitly set allow_ref on the Parameter definition to declare whether references should be resolved or not.
  super().__init__(**params)


In [53]:
dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'ee7b3250-1723-4463-ab44-b759db82786a': {'version…

In [61]:

def prices(df):
    
    # Crear una máscara de filtro para la fecha seleccionada
    #mask = (df['date'] == date)
    #df_filtered = df.loc[mask]

    df_filtered = df.groupby('datetime')['value'].sum().reset_index()


    print(df_filtered['value'].describe())

prices(df_E_es)

count    8760.000000
mean       97.723726
std        40.100838
min         6.190000
25%        74.765000
50%       101.265000
75%       124.850000
max       251.510000
Name: value, dtype: float64


In [76]:
# Cargar el archivo Excel
file_path = 'Datos/Economico/PrecioEuropa_2023_h.csv'
#df = pd.read_excel(file_path)
df_E_eu = pd.read_csv(file_path, delimiter=';')

In [77]:
df_E_eu['name'].unique()

array(['Precio mercado SPOT Diario Portugal',
       'Precio mercado SPOT Diario Francia',
       'Precio mercado SPOT Diario España',
       'Precio mercado SPOT Diario Reino Unido',
       'Precio mercado SPOT Diario Italia',
       'Precio mercado SPOT Diario Alemania',
       'Precio mercado SPOT Diario Bélgica',
       'Precio mercado SPOT Diario Países Bajos',
       'Término de facturación de energía activa del PVPC 2.0TD Península',
       'Término de facturación de energía activa del PVPC 2.0TD Canarias',
       'Término de facturación de energía activa del PVPC 2.0TD Baleares',
       'Término de facturación de energía activa del PVPC 2.0TD Ceuta',
       'Término de facturación de energía activa del PVPC 2.0TD Melilla'],
      dtype=object)

In [78]:
# Convertir la columna datetime a tipo datetime
df_E_eu['datetime'] = pd.to_datetime(df_E_eu['datetime'], utc = True) #, utc = True

# Convertir la columna name a tipo categoría
df_E_eu['name'] = df_E_eu['name'].astype('category')


df_E_eu = df_E_eu.drop(df_E_eu[df_E_eu['id']== 1001].index)
df_E_eu['name'] = df_E_eu['name'].str.slice(27).str.capitalize()
df_E_eu['date'] = df_E_eu['datetime'].dt.date

In [123]:
@pn.depends(date_slider.param.value)
def get_plot_price_pais(date):
     # Load and format the data
     df = df_E_eu # define df
     ##df[‘date’] = pd.to_datetime(df[‘date’])
     # create date filter using values from the range slider
     # store the first and last date range slider value in a var
     start_date = date_slider.value
     # create filter mask for the dataframe
     mask = (df['date'] == start_date)
     df = df.loc[mask] # filter the dataframe
    
     df['datetime'] = df['datetime'] + dt.timedelta(hours=0, minutes=30)
     
     order = ['Alemania','Bélgica','España','Francia','Italia','Países bajos','Portugal','Reino unido']
     # Convertir la columna 'category' a un tipo categórico con el orden definido
     df['name'] = pd.Categorical(df['name'], categories=order, ordered=True)
     
     # Ordenar el DataFrame según la columna 'category'
     df = df.sort_values('name')


     selection = alt.selection_point(fields=['name'], bind='legend')

     # create the Altair chart object
     """
     chart = alt.Chart(df).mark_line(interpolate='step').encode(
               x=alt.X('datetime:T', title='Hora del día'),  # Eje X: Hora del día
               y=alt.Y('value:Q', title='Precio por MWh (€/MWh)', scale=alt.Scale(domain=[-10, 250])),
               color=alt.Color('name:N', title='País'), 
               opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
               ).properties(
               title='Precio electricidad por paises',
               width=700,
               height=400
               ).add_params(selection)
     """
     # Create a selection that chooses the nearest point & selects based on x-value
     nearest = alt.selection_point(nearest=True, on="pointerover",
                              fields=["datetime"], empty=False)
     
     # The basic line
     line = alt.Chart(df).mark_line(interpolate='step').encode(
               x=alt.X('datetime:T', title='Hora del día'),  # Eje X: Hora del día
               y=alt.Y('value:Q', title='Precio por MWh (€/MWh)', scale=alt.Scale(domain=[-10, 250])),
               color=alt.Color('name:N', title='País'), 
               opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
               ).properties(
               title='Precio electricidad por paises',
               width=700,
               height=400
               ).add_params(selection)

     # Draw points on the line, and highlight based on selection
     points = line.mark_point().encode(
          opacity=alt.condition(nearest, alt.value(1), alt.value(0)))
     
     # Draw a rule at the location of the selection
     rules = alt.Chart(df).transform_pivot(
        "name",
        value="value",
        groupby=["datetime"]).mark_rule(color="gray").encode(
         x="datetime",
         opacity=alt.condition(nearest, alt.value(0.3), alt.value(0)),
         tooltip=[alt.Tooltip(c, type="quantitative") for c in order],
     ).add_params(nearest)

     # Put the five layers into a chart and bind the data
     chart = alt.layer(
         line, points, rules
     ).properties(
         width=600, height=300
     )

     return chart

In [124]:
title = '## Desglose del precio de la electricidad'
subtitle = 'Este dashboard permite al usuario seleccionar\n el día para el que se quieren visualizar los datos.\n\n Además, también se puede seleccionar la tecnología\n para resaltarla en el gráfico.'

dashboard = pn.Row(get_plot_price_pais,
                   pn.Column(title, subtitle, date_slider))

/tmp/ipykernel_3311363/1251896303.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['datetime'] = df['datetime'] + dt.timedelta(hours=0, minutes=30)
/tmp/ipykernel_3311363/1251896303.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['name'] = pd.Categorical(df['name'], categories=order, ordered=True)
/usuarios/jos/.conda/envs/jos_prueba/lib/python3.8/site-packages/panel/viewable.py:298: ParamFutureWarning: Parameter 'object' on <class 'panel.pane.base.PaneBase'> is being given a valid parameter 

In [125]:
dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'fcf716f4-6d8b-40bd-ab52-8712938156e3': {'version…

In [62]:
import altair as alt
from vega_datasets import data

counties = alt.topo_feature(data.us_10m.url, 'counties')
source = data.unemployment.url

alt.Chart(counties).mark_geoshape().encode(
    color='rate:Q'
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(source, 'id', ['rate'])
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
)

alt.Chart(...)

In [63]:
counties

UrlData({
  format: TopoDataFormat({
    feature: 'counties',
    type: 'topojson'
  }),
  url: 'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/us-10m.json'
})

## 2.3 Datos de emisiones

### 2.3.1. 

In [2]:
# Cargar el archivo Excel
file_path = 'Datos/Emisiones/output/CI_bottom_up_method.csv'
#df = pd.read_excel(file_path)
df_emis = pd.read_csv(file_path, delimiter=',')


In [3]:
# Convertir la columna datetime a tipo datetime
df_emis['datetime'] = pd.to_datetime(df_emis['datetime'], utc = True) #, utc = True
df_emis['date'] = df_emis['datetime'].dt.date

In [4]:
df_emis[['datetime','date','BE','DE','ES','FR','GB','IT','NL','PT']]

,datetime,date,BE,DE,ES,FR,GB,IT,NL,PT
0,2023-01-01 00:00:00+00:00,2023-01-01,104.679623,199.522307,116.313392,21.426519,173.306847,414.529444,115.706888,125.564642
1,2023-01-01 01:00:00+00:00,2023-01-01,109.305971,198.561804,128.916633,21.828823,187.855483,406.603034,115.292381,129.650644
2,2023-01-01 02:00:00+00:00,2023-01-01,114.954122,195.428854,148.118548,22.131552,182.839003,404.245875,113.533090,121.907224
3,2023-01-01 03:00:00+00:00,2023-01-01,114.036683,195.845125,148.488932,22.623573,177.384454,395.961700,112.353861,126.724787
4,2023-01-01 04:00:00+00:00,2023-01-01,113.386372,193.798527,149.926876,22.618121,160.737181,394.294617,112.358109,127.082265
...,...,...,...,...,...,...,...,...,...,...
8755,2023-12-31 19:00:00+00:00,2023-12-31,227.313382,338.216391,235.631276,34.205501,232.652273,330.166842,489.073799,198.556560
8756,2023-12-31 20:00:00+00:00,2023-12-31,222.388394,325.576135,237.980872,25.609419,202.723269,329.123537,485.371634,222.061191
8757,2023-12-31 21:00:00+00:00,2023-12-31,209.481579,313.343889,237.493332,25.529700,164.893157,336.090406,482.631494,276.423569
8758,2023-12-31 22:00:00+00:00,2023-12-31,202.789259,293.122606,239.756390,23.398160,158.442344,332.356797,494.644721,312.591723


In [5]:
df_emis = pd.melt(df_emis, id_vars=['datetime', 'date'], var_name='name', value_name='value')


In [6]:
df_emis_avg = df_emis.copy().groupby(['date','name']).agg({'value': 'mean'}).reset_index()

In [7]:
df_emis_avg['date'] = pd.to_datetime(df_emis_avg['date'])
df_emis_avg

,date,name,value
0,2023-01-01,AT,99.115130
1,2023-01-01,BE,117.320993
2,2023-01-01,BG,458.277457
3,2023-01-01,CH,0.000000
4,2023-01-01,CY,673.295525
...,...,...,...
10580,2023-12-31,RO,339.897335
10581,2023-12-31,RS,909.331237
10582,2023-12-31,SE,63.584709
10583,2023-12-31,SI,249.383931


In [180]:
alt.Chart(df_emis_avg[df_emis_avg['name']=='ES'], title="Daily Max Temperatures (C) in Seattle, WA").mark_rect().encode(
    alt.X("date(date):O").title("Day").axis(format="%e", labelAngle=0),
    alt.Y("month(date):O").title("Month"),
    alt.Color("value").title(None),
    tooltip=[
        alt.Tooltip("monthdate(date)", title="Date"),
        alt.Tooltip("value", title="Max Temp"),
    ],
).configure_view(
    step=13,
    strokeWidth=0
).configure_axis(
    domain=False)

alt.Chart(...)

In [181]:
alt.Chart(df_emis_avg[df_emis_avg['name']=='DE'], title="Daily Max Temperatures (C) in Seattle, WA").mark_rect().encode(
    alt.X("date(date):O").title("Day").axis(format="%e", labelAngle=0),
    alt.Y("month(date):O").title("Month"),
    alt.Color("value").title(None),
    tooltip=[
        alt.Tooltip("monthdate(date)", title="Date"),
        alt.Tooltip("value", title="Max Temp"),
    ],
).configure_view(
    step=13,
    strokeWidth=0
).configure_axis(
    domain=False)

alt.Chart(...)

In [8]:

source = data.iris()

base = alt.Chart(source)
base_bar = base.mark_bar(opacity=0.3, binSpacing=0)

xscale = alt.Scale(domain=(4.0, 8.0))
yscale = alt.Scale(domain=(1.9, 4.55))

points = base.mark_circle().encode(
    alt.X("sepalLength").scale(xscale),
    alt.Y("sepalWidth").scale(yscale),
    color="species",
)

top_hist = (
    base_bar
    .encode(
        alt.X("sepalLength:Q")
            # when using bins, the axis scale is set through
            # the bin extent, so we do not specify the scale here
            # (which would be ignored anyway)
            .bin(maxbins=20, extent=xscale.domain).stack(None).title(""),
        alt.Y("count()").stack(None).title(""),
        alt.Color("species:N"),
    )
    .properties(height=60)
)

right_hist = (
    base_bar
    .encode(
        alt.Y("sepalWidth:Q")
            .bin(maxbins=20, extent=yscale.domain)
            .stack(None)
            .title(""),
        alt.X("count()").stack(None).title(""),
        alt.Color("species:N"),
    )
    .properties(width=60)
)

top_hist & (points | right_hist)

ModuleNotFoundError: No module named 'vega_datasets'

In [142]:
import altair as alt
from vega_datasets import data

source = data.seattle_weather()

alt.Chart(source, title="Daily Max Temperatures (C) in Seattle, WA").mark_rect().encode(
    alt.X("date(date):O").title("Day").axis(format="%e", labelAngle=0),
    alt.Y("month(date):O").title("Month"),
    alt.Color("max(temp_max)").title(None),
    tooltip=[
        alt.Tooltip("monthdate(date)", title="Date"),
        alt.Tooltip("max(temp_max)", title="Max Temp"),
    ],
).configure_view(
    step=13,
    strokeWidth=0
).configure_axis(
    domain=False)

print(source)

           date  precipitation  temp_max  temp_min  wind  weather
0    2012-01-01            0.0      12.8       5.0   4.7  drizzle
1    2012-01-02           10.9      10.6       2.8   4.5     rain
2    2012-01-03            0.8      11.7       7.2   2.3     rain
3    2012-01-04           20.3      12.2       5.6   4.7     rain
4    2012-01-05            1.3       8.9       2.8   6.1     rain
...         ...            ...       ...       ...   ...      ...
1456 2015-12-27            8.6       4.4       1.7   2.9      fog
1457 2015-12-28            1.5       5.0       1.7   1.3      fog
1458 2015-12-29            0.0       7.2       0.6   2.6      fog
1459 2015-12-30            0.0       5.6      -1.0   3.4      sun
1460 2015-12-31            0.0       5.6      -2.1   3.5      sun

[1461 rows x 6 columns]
